In [ ]:
import os
import math
import torch
import random
import shutil
import pickle
import pprint
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import models, transforms
from transformers import ViTFeatureExtractor, ViTModel, ViTImageProcessor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [2]:
warnings.filterwarnings("ignore")
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
def is_empty(channel):
    return np.sum(channel) == 0

def map_values(value, old_min, old_max, new_min, new_max):
    return int(new_min + (value - old_min) * (new_max - new_min) / (old_max - old_min))

vectorized_map_values = np.vectorize(map_values)

In [15]:
# Define the path to the folder
new_min = 0
new_max = 71
mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
data_names = []

for mode in mode_list:
    for folder in os.listdir('./original_data'):
        if os.path.isdir('./original_data' + '/'+folder):
            folder_path = './original_data'+'/'+folder
            # Get a list of all PNG files in the folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.png')]
            files_code = list(set([f[5:9] for f in files]))
            for code in files_code:
                data = np.zeros((72, 232, 465))
                temp_dict = {}
                temp_list = []
                file_names = []
                for file in files:
                    if code == file[5:9] and mode in file:
                        index = 0
                        if len(file) == 33:
                            index = int(file[26:29])
                        elif len(file) == 32:
                            index = int(file[26:28])
                        temp_list.append(index)
                        file_names.append(file)
                # temp_list.sort()
                if len(temp_list) == 0:
                    continue
                # print(file_names)
                old_min = min(temp_list)
                old_max = max(temp_list)
                new_values = vectorized_map_values(np.array(temp_list), old_min, old_max, new_min, new_max)
                for i in range(len(new_values)):
                    temp_dict[new_values[i]] = file_names[i]
                for key in sorted(temp_dict.keys()):
                    data[key, :, :] = np.array(Image.open(os.path.join(folder_path, temp_dict[key])))[:, :, 0]

                # Find the indices of the channels with data
                data_indices = [i for i in range(data.shape[0]) if not is_empty(data[i])]

                # Fill the empty channels with gradient
                for i in range(data.shape[0]):
                    if is_empty(data[i]):
                        # Find the nearest channels with data before and after the empty channel
                        before = max([idx for idx in data_indices if idx < i], default=None)
                        after = min([idx for idx in data_indices if idx > i], default=None)
                        
                        if before is not None and after is not None:
                            # Interpolate between the before and after channels
                            gradient = (data[after] - data[before]) / (after - before)
                            data[i] = data[before] + gradient * (i - before)
                        elif before is not None:
                            # If there is no after channel, use the before channel
                            data[i] = data[before]
                        elif after is not None:
                            # If there is no before channel, use the after channel
                            data[i] = data[after]

                file_name = os.path.join('combined_data', folder, code+'_'+mode+'.npy')
                data_names.append(file_name)
                np.save(file_name, data)

In [ ]:
milddemented = [ele[27:31] for ele in data_names if 'MildDemented' in ele and 'VeryMildDemented' not in ele]
moderatedemented = [ele[31:35] for ele in data_names if 'ModerateDemented' in ele]
nonedemented = [ele[26:30] for ele in data_names if 'NonDemented' in ele]
verymildlydemented = [ele[31:35] for ele in data_names if 'VeryMildDemented' in ele]

In [ ]:
data_frame = pd.read_csv('oasis_cross-sectional.csv')
data_frame.drop('Delay', axis=1, inplace=True)
data_frame = data_frame.dropna()

records_codes = [ele[5:9] for ele in list(data_frame['ID'])]
records_codes_set = set(records_codes)
files_codes = [ele[31:35] for ele in data_names]
files_codes_set = set(files_codes)

patients_codes_set = records_codes_set.intersection(files_codes_set)
# save the data of the patients
with open('patients_codes_set.pkl', 'wb') as f:
    pickle.dump(patients_codes_set, f)

print("Set saved to patients_codes_set.pkl")

In [ ]:
with open('patients_codes_set.pkl', 'rb') as f:
    patients_codes_set = pickle.load(f)

print("Set loaded from patients_codes_set.pkl:", patients_codes_set)

In [ ]:
data_frame[:2]
len(patients_codes_set)

In [ ]:
# choose test items
# Set the random seed for reproducibility
with open('patients_codes_set.pkl', 'rb') as f:
    patients_codes_set = pickle.load(f)

random.seed(42)

# Define the folders
folders = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
base_path = 'test_data'  # Replace with the path to your data

# Function to randomly select and delete files
def select_and_delete_files(folder_path, percentage):
    # List all files in the folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    # Calculate the number of files to keep
    num_files_to_keep = int(len(files) * percentage)
    
    # Randomly select files to keep
    files_to_keep = random.sample(files, num_files_to_keep)
    
    # Delete the remaining files
    for file in files:
        # print(file)
        if file not in files_to_keep and file[:4] in patients_codes_set:
            os.remove(os.path.join(folder_path, file))
            # print(f"Deleted: {os.path.join(folder_path, file)}")

# # Process each folder
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    select_and_delete_files(folder_path, 0.8)

print("Process complete.")

In [13]:
# choose test items
# Set the random seed for reproducibility
with open('patients_codes_set.pkl', 'rb') as f:
    patients_codes_set = pickle.load(f)

random.seed(42)

# Define the folders
folders = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
base_path = 'train_data'  # Replace with the path to your data

# Function to randomly select and delete files
def select_and_delete_files(folder_path, percentage):
    # List all files in the folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    # Calculate the number of files to keep
    num_files_to_keep = int(len(files) * percentage)
    
    # Randomly select files to keep
    files_to_keep = random.sample(files, num_files_to_keep)
    
    # Delete the remaining files
    for file in files:
        # print(file)
        if file not in files_to_keep or file[:4] in patients_codes_set:
            os.remove(os.path.join(folder_path, file))
            # print(f"Deleted: {os.path.join(folder_path, file)}")

# # Process each folder
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    select_and_delete_files(folder_path, 1)

print("Process complete.")

Process complete.


In [ ]:
# regression model
# Initialize the feature extractor and the model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.linear_layer = nn.Linear(vit_model.config.hidden_size, 3)
        self.linear_layer.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(8, 5)
        self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 3)
        self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.vit_model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state.mean(dim=1)
        final_output = self.linear_layer(pooled_output)
        # print(final_output.shape, x.shape)
        final_output = torch.cat((final_output, x), 1)
        # print(final_output.shape)
        final_output = self.relu(final_output)
        final_output = self.out_1(final_output)
        final_output = self.relu(final_output)
        final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
criterion = nn.MSELoss()


In [22]:
regression_dataframe = pd.DataFrame(columns=['Type', 'model', 'MMSE MAE', 'CDR MAE', 'SES MAE', 'MMSE MSE', 'CDR MSE', 'SES MSE', 'MMSE R2', 'CDR R2', 'SES R2', 'MMSE RMSE', 'CDR RMSE', 'SES RMSE'])

test_regression_dataframe = pd.DataFrame(columns=['Type', 'model', 'MMSE MAE', 'CDR MAE', 'SES MAE', 'MMSE MSE', 'CDR MSE', 'SES MSE', 'MMSE R2', 'CDR R2', 'SES R2', 'MMSE RMSE', 'CDR RMSE', 'SES RMSE'])

In [ ]:
# regression MR1_1
model = CombinedModel(vit_model).to(device)
model_load_path = 'models/MR1_1_reg.pth'
model.load_state_dict(torch.load(model_load_path))

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(10001):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
                x = torch.tensor(x).float()
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor, x)
                loss = criterion(outputs, torch.tensor(y).float())
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((outputs.detach().numpy(), y))
    if epoch % 50 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])
results.to_csv('results/'+mode+'_'+'results_reg.csv', index=False)

model_save_path = 'models/'+mode+'_'+'reg.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [ ]:
print(model)

In [ ]:
mode = 'MR1_1'
results = pd.read_csv('results/'+mode+'_'+'results_reg.csv')

# adding MR1_1 results to the dataframe
mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_1', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

regression_dataframe = pd.concat([regression_dataframe, new_data], ignore_index=True)

new_data

In [ ]:
# regression MR1_1 test
model = CombinedModel(vit_model).to(device)
model_load_path = 'models/MR1_1_reg.pth'
model.load_state_dict(torch.load(model_load_path))

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            loss = criterion(outputs, torch.tensor(y).float())
            running_loss += loss.item()
            results.append((outputs.detach().numpy(), y))
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])

results.to_csv('test_results/'+mode+'_'+'results_reg.csv', index=False)

# adding MR1_2 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_1', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

test_regression_dataframe = pd.concat([test_regression_dataframe, new_data], ignore_index=True)

test_regression_dataframe

In [ ]:
# regression MR1_2
model = CombinedModel(vit_model).to(device)
model_load_path = 'models/MR1_1_reg.pth'
model.load_state_dict(torch.load(model_load_path))
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(501):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
                x = torch.tensor(x).float()
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor, x)
                loss = criterion(outputs, torch.tensor(y).float())
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((outputs.detach().numpy(), y))
    if epoch == 500:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])
results.to_csv('results/'+mode+'_'+'results_reg.csv', index=False)

model_save_path = 'models/'+mode+'_'+'reg.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [27]:
# adding MR1_2 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_2', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

regression_dataframe = pd.concat([regression_dataframe, new_data], ignore_index=True)

In [28]:
# regression MR1_2 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            loss = criterion(outputs, torch.tensor(y).float())
            running_loss += loss.item()
            results.append((outputs.detach().numpy(), y))
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])

results.to_csv('test_results/'+mode+'_'+'results_reg.csv', index=False)

# adding MR1_2 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_2', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

test_regression_dataframe = pd.concat([test_regression_dataframe, new_data], ignore_index=True)

In [ ]:
# regression MR1_3
model = CombinedModel(vit_model).to(device)
model_load_path = 'models/MR1_1_reg.pth'
model.load_state_dict(torch.load(model_load_path))
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(501):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
                x = torch.tensor(x).float()
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor, x)
                loss = criterion(outputs, torch.tensor(y).float())
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((outputs.detach().numpy(), y))
    if epoch == 500:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])
results.to_csv('results/'+mode+'_'+'results_reg.csv', index=False)

model_save_path = 'models/'+mode+'_'+'reg.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval() # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [30]:
# adding MR1_3 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_3', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

regression_dataframe = pd.concat([regression_dataframe, new_data], ignore_index=True)

In [31]:
# regression MR1_3 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            loss = criterion(outputs, torch.tensor(y).float())
            running_loss += loss.item()
            results.append((outputs.detach().numpy(), y))
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])

results.to_csv('test_results/'+mode+'_'+'results_reg.csv', index=False)

# adding MR1_2 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_3', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

test_regression_dataframe = pd.concat([test_regression_dataframe, new_data], ignore_index=True)

In [ ]:
# regression MR1_4
model = CombinedModel(vit_model).to(device)
model_load_path = 'models/MR1_1_reg.pth'
model.load_state_dict(torch.load(model_load_path))
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(501):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
                x = torch.tensor(x).float()
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor, x)
                loss = criterion(outputs, torch.tensor(y).float())
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((outputs.detach().numpy(), y))
    if epoch == 500:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])
results.to_csv('results/'+mode+'_'+'results_reg.csv', index=False)

model_save_path = 'models/'+mode+'_'+'reg.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [33]:
# adding MR1_4 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_4', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

regression_dataframe = pd.concat([regression_dataframe, new_data], ignore_index=True)

In [34]:
# regression MR1_4 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            loss = criterion(outputs, torch.tensor(y).float())
            running_loss += loss.item()
            results.append((outputs.detach().numpy(), y))
results = np.array(results).reshape(-1, 6)
results = pd.DataFrame(results, columns=['Predicted MMSE', 'Predicted CDR', 'Predicted SES', 'True MMSE', 'True CDR', 'True SES'])

results.to_csv('test_results/'+mode+'_'+'results_reg.csv', index=False)

# adding MR1_2 results to the dataframe

mae_MMSE = mean_absolute_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MAE: {mae_MMSE}')
mae_CDR = mean_absolute_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MAE: {mae_CDR}')
mae_SES = mean_absolute_error(results['True SES'], results['Predicted SES'])
# print(f'SES MAE: {mae_SES}')

mse_MMSE = mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE MSE: {mse_MMSE}')
mse_CDR = mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR MSE: {mse_CDR}')
mse_SES = mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES MSE: {mse_SES}')

r2_MMSE = r2_score(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE R2: {r2_MMSE}')
r2_CDR = r2_score(results['True CDR'], results['Predicted CDR'])
# print(f'CDR R2: {r2_CDR}')
r2_SES = r2_score(results['True SES'], results['Predicted SES'])
# print(f'SES R2: {r2_SES}')

rmse_MMSE = root_mean_squared_error(results['True MMSE'], results['Predicted MMSE'])
# print(f'MMSE RMSE: {rmse_MMSE}')
rmse_CDR = root_mean_squared_error(results['True CDR'], results['Predicted CDR'])
# print(f'CDR RMSE: {rmse_CDR}')
rmse_SES = root_mean_squared_error(results['True SES'], results['Predicted SES'])
# print(f'SES RMSE: {rmse_SES}')

new_data = pd.DataFrame([{'Type': 'Regression', 'model': 'MR1_4', 'MMSE MAE': mae_MMSE, 'CDR MAE': mae_CDR, 'SES MAE': mae_SES, 'MMSE MSE': mse_MMSE, 'CDR MSE': mse_CDR, 'SES MSE': mse_SES, 'MMSE R2': r2_MMSE, 'CDR R2': r2_CDR, 'SES R2': r2_SES, 'MMSE RMSE': rmse_MMSE, 'CDR RMSE': rmse_CDR, 'SES RMSE': rmse_SES}])

test_regression_dataframe = pd.concat([test_regression_dataframe, new_data], ignore_index=True)

In [ ]:
# classification model
# Initialize the feature extractor and the model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel_classfication(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel_classfication, self).__init__()
        self.vit_model = vit_model
        self.linear_layer = nn.Linear(vit_model.config.hidden_size, 3)
        self.linear_layer.weight.data.normal_(mean=0.0, std=0.02)
        self.out = nn.Linear(3, 4)
        self.relu = nn.ReLU()
        self.out.weight.data.normal_(mean=0.0, std=0.02)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Preprocess the input tensor
        x = x.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=x, return_tensors="pt")
        outputs = self.vit_model(**inputs)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state.mean(dim=1)
        final_output = self.linear_layer(pooled_output)
        final_output = self.relu(final_output)
        final_output = self.out(final_output)
        final_output = self.softmax(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel_classfication(vit_model).to(device)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()

In [36]:
classification_dataframe = pd.DataFrame(columns=['Type', 'model', 'Accuracy', 'Precision', 'Recall', 'True positive', 'Sensitivity', 'Specificity', 'F1 Score'])

test_classification_dataframe = pd.DataFrame(columns=['Type', 'model', 'Accuracy', 'Precision', 'Recall', 'True positive', 'Sensitivity', 'Specificity', 'F1 Score'])

In [ ]:
# classification MR1_1
model = CombinedModel_classfication(vit_model).to(device)
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(10001):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor)
                target = torch.tensor([label]).to(device)
                loss = criterion(outputs, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((np.argmax(outputs.detach().numpy()), label))
    if epoch == 500:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('results/'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'models/'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [38]:
# adding MR1_1 results to the dataframe

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_1', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

In [39]:
# classification MR1_1 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((np.argmax(outputs.detach().numpy()), label))


# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('test_results/'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_1', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

test_classification_dataframe = pd.concat([test_classification_dataframe, new_data], ignore_index=True)


In [ ]:
# classification MR1_2
model = CombinedModel_classfication(vit_model).to(device)
model_load_path = 'models/MR1_1_class.pth'
model.load_state_dict(torch.load(model_load_path))

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(501):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor)
                target = torch.tensor([label]).to(device)
                loss = criterion(outputs, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 == 0:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('results/'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'models/'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [41]:
# adding MR1_2 results to the dataframe

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_2', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

In [42]:
# classification MR1_2 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((np.argmax(outputs.detach().numpy()), label))


# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('test_results/'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_2', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

test_classification_dataframe = pd.concat([test_classification_dataframe, new_data], ignore_index=True)


In [ ]:
# classification MR1_3
model = CombinedModel_classfication(vit_model).to(device)
model_load_path = 'models/MR1_1_class.pth'
model.load_state_dict(torch.load(model_load_path))

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(101):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor)
                target = torch.tensor([label]).to(device)
                loss = criterion(outputs, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((np.argmax(outputs.detach().numpy()), label))
    if epoch == 100:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('results/'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'models/'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [44]:
# adding MR1_3 results to the dataframe

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_3', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

In [45]:
# classification MR1_3 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((np.argmax(outputs.detach().numpy()), label))


# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('test_results/'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_3', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

test_classification_dataframe = pd.concat([test_classification_dataframe, new_data], ignore_index=True)


In [ ]:
# classification MR1_4
model = CombinedModel_classfication(vit_model).to(device)
model_load_path = 'models/MR1_1_class.pth'
model.load_state_dict(torch.load(model_load_path))

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(101):
    running_loss = 0.0
    results = []
    for folder in os.listdir('./combined_data'):
        if os.path.isdir('./combined_data' + '/'+folder):
            folder_path = './combined_data'+'/'+folder
            files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
            if folder == 'NonDemented':
                label = 0
            elif folder == 'VeryMildDemented':
                label = 1
            elif folder == 'MildDemented':
                label = 2
            elif folder == 'ModerateDemented':
                label = 3
            for f in files :
                data = np.load(os.path.join(folder_path, f))
                code = f[0:4]
                y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
                model.train()
                optimizer.zero_grad()
                input_tensor = torch.clamp(torch.tensor(data), 0, 1)
                outputs = model(input_tensor)
                target = torch.tensor([label]).to(device)
                loss = criterion(outputs, target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                results.append((np.argmax(outputs.detach().numpy()), label))
    if epoch == 100:
        print(f'Epoch {epoch} loss: {running_loss}')

# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('results/'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'models/'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

# Load the model state
# model_load_path = model_save_path
# model.load_state_dict(torch.load(model_load_path))
# model.eval()  # Set the model to evaluation mode
# print(f"Model loaded from {model_load_path}")

In [47]:
# adding MR1_4 results to the dataframe

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_4', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

In [48]:
# classification MR1_4 test

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((np.argmax(outputs.detach().numpy()), label))


# convert the results to a numpy array
results = np.array(results).reshape(-1, 2)
results = pd.DataFrame(results, columns=['Predicted class', 'True class'])
results.to_csv('test_results/'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': 'MR1_4', 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

test_classification_dataframe = pd.concat([test_classification_dataframe, new_data], ignore_index=True)


In [54]:
regression_dataframe.to_csv('results/regression_results.csv', index=False)
classification_dataframe.to_csv('results/classification_results.csv', index=False)
test_regression_dataframe.to_csv('test_results/test_regression_results.csv', index=False)
test_classification_dataframe.to_csv('test_results/test_classification_results.csv', index=False)

In [ ]:
# image_path = 'original_data/MildDemented/OAS1_0028_MR1_1.nii_slice_113.png'
# image = Image.open(image_path)
# img_array = np.array(image)
# img_array = img_array[:, :, :3] / 255.0
# img_array = torch.tensor(img_array).permute(2, 0, 1)
# y = feature_extractor(images=img_array, return_tensors="pt")['pixel_values'].squeeze()
# plt.imshow(img_array[0, :, :], cmap='gray')
# plt.show()
# plt.imshow(y[2, :, :], cmap='gray')
# plt.show()

In [ ]:
# intermediate_outputs = []

# def hook(module, input, output):
#     intermediate_outputs.append(output)
# # Register the hook
# hook_handle = model.vit_model.encoder.layer[0].output.register_forward_hook(hook)

# # Get the final output
# final_output = model(img_array)

# # Remove the hook
# hook_handle.remove()

# print("Final output shape:", final_output.shape)
# print("Intermediate output shape:", intermediate_outputs[0].shape)
# plt.imshow(intermediate_outputs[0][0, :, :].detach().numpy(), cmap='gray')

In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

running_loss = 0.0
results = []
for folder in os.listdir('./test'):
    if os.path.isdir('./test' + '/'+folder):
        folder_path = './test'+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f and f[0:4] in patients_codes_set]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((np.argmax(outputs.detach().numpy()), label))